In [2]:
# Imports
import random
import string
from br_nome_gen import pessoa_random
from psycopg2 import connect
import time

In [3]:
# Criação da classe de conexão com o banco de dados
class ConnectDB:

    def __init__(self):

        self.con = connect(
            user='postgres',
            password='postgres',
            host='localhost',
            port='5432',
            dbname='bd_Nubank_Cripto',
        )
        self.cur = self.con.cursor()

    def lista_cpf(self):
        query = 'SELECT cpf_cliente FROM cliente;'
        try:
            self.cur.execute(query)
        except Exception as e:
            self.con.rollback()
            print("Failed")
        else:
            self.con.commit()
            #print("Success")
        return self.cur.fetchall()
    
    def lista_wallet(self):
        query = 'SELECT cod_wallet FROM wallet;'
        try:
            self.cur.execute(query)
        except Exception as e:
            self.con.rollback()
            print("Failed")
        else:
            self.con.commit()
            #print("Success")
        return self.cur.fetchall()
    
    def lista_criptomoeda(self):
        query = 'SELECT cod_criptomoeda FROM criptomoeda;'
        try:
            self.cur.execute(query)
        except Exception as e:
            self.con.rollback()
            print("Failed")
        else:
            self.con.commit()
            #print("Success")
        return self.cur.fetchall()
    
    def lista_trade(self):
        query = 'SELECT hash_id FROM trade;'
        try:
            self.cur.execute(query)
        except Exception as e:
            self.con.rollback()
            print("Failed")
        else:
            self.con.commit()
            #print("Success")
        return self.cur.fetchall()

    def count_clientes(self):
        query = 'SELECT COUNT(*) FROM cliente;'
        self.cur.execute(query)
        return self.cur.fetchone()[0]

    def insert_clientes(self, data):
        query = '''INSERT INTO cliente
        (cpf_cliente, nome_cliente, data_nascimento, contato_celular, email) 
        VALUES (%s, %s, %s, %s, %s);'''
        try:
            self.cur.execute(query, data)
        except Exception as e:
            self.con.rollback()
            #print("Failed")
        else:
            self.con.commit()
            # print("Success")

    def insert_wallet(self, data):
        query = '''INSERT INTO wallet 
            (saldo_wallet, cpf_cliente)
            VALUES(%s, %s); '''
        try:
            self.cur.execute(query, data)
        except Exception as e:
            self.con.rollback()
            print("Failed")
        else:
            self.con.commit()
            # print("Success")

    def insert_trade(self, data):
        query = '''INSERT INTO trade
	            (hash_id, tipo_movimentacao, data_trade, hora_trade)
	            VALUES (%s, %s, %s, %s);'''
        try:
            self.cur.execute(query, data)
        except Exception as e:
            self.con.rollback()
            print("Failed")
        else:
            self.con.commit()
            #print("Success")

    def insert_criptomoeda(self, data):
        query = ''' INSERT INTO criptomoeda
	            (nome_cripto, sigla_cripto, valor_atual)
	            VALUES (%s, %s, %s);'''
        try:
            self.cur.execute(query, data)
        except Exception as e:
            self.con.rollback()
            print("Failed")
        else:
            self.con.commit()
            #print("Success")

    def insert_cripto_na_carteira(self, data):
        query = ''' INSERT INTO criptos_na_carteira(
	            quantidade, cod_criptomoeda, cod_wallet)
	            VALUES (%s, %s, %s);'''
        try:
            self.cur.execute(query, data)
        except Exception as e:
            self.con.rollback()
            print("Failed")
        else:
            self.con.commit()
            #print("Success")

    def insert_trade_realizada(self, data):
        query = '''INSERT INTO trade_realizada
                (hash_id, cod_criptomoeda, cod_wallet, cpf_cliente)
                VALUES (%s, %s, %s, %s); '''
        try:
            self.cur.execute(query, data)
        except Exception as e:
            self.con.rollback()
            print("Failed")
        else:
            self.con.commit()
            #print("Success")

    def find_wallet_by_cliente(self, rowid):
        query = '''SELECT cod_wallet FROM wallet WHERE cpf_cliente = %s'''
        try:
            self.cur.execute(query, (rowid,))
        except Exception as e:
            self.con.rollback()
            print("Failed")
        else:
            self.con.commit()
            return self.cur.fetchone()
            #print("Success")



### Gerando registros para a tabela Cliente

In [4]:
# função geradora de cpf
def gerador_cpf():
    retorno = ""
    for item in range(11):
        x = random.choice(string.digits);
        retorno += x
    return retorno

In [5]:
# Função geradora de nome e emails
# a geração desses 2 campos juntas foi necessaria para facilitar a criação do 'email' com o mesmo nome randomico presente no campo 'nome'
# por isso este metodo retorna um dicionario com 2 chaves (nome, email) 
def gerador_nome_email():
    pessoa = pessoa_random()
    email = f"{pessoa.nome.split(' ')[0]}@gmail.com"
    retorno = {'nome':pessoa.nome, 'email':email}
    return retorno

In [6]:
def gerador_data_nascimento():
    ano = random.randint(1950, 2007)
    mes = random.randint(1, 12)

    if mes == 1 or mes == 3 or mes == 5 or mes == 7 or mes == 8 or mes == 10 or mes == 12:
        dia = random.randint(1, 31)
    elif mes == 4 or mes == 6 or mes == 9 or mes == 11:
        dia = random.randint(1, 30)
    elif mes == 2:
        dia = random.randint(1, 28)
    return f"{ano}-{mes}-{dia}"

In [7]:
def gerador_celular():
    inicio = '99'
    resto = ""
    for item in range(7):
        x = random.choice(string.digits);
        resto += x
    return f"{inicio}{resto}"
    

In [8]:
# Criando a conexão com o banco.
database = ConnectDB()
# Alterar a quantidade de repetições
QTD_REGISTROS = 1
for item in range (QTD_REGISTROS):
    # Criando o registro para a inserção
    nome_e_email = gerador_nome_email()
    user = (gerador_cpf(), nome_e_email['nome'], gerador_data_nascimento(), gerador_celular(), nome_e_email['email'])
    # Inserindo um registro tabela.
    database.insert_clientes(data=user)

# Fechando conexão com o banco.
database.con.close()

### Gerando registros para a tabela Wallet

In [9]:
database = ConnectDB()
cpfs_clientes = database.lista_cpf()
database.con.close()

In [10]:
database = ConnectDB()
QTD_REGISTROS = 1
for i in range (QTD_REGISTROS):
    cpf_escolhido = random.choice(cpfs_clientes)
    saldo = float(random.randint(1, 1000000))
    wallet = (saldo, cpf_escolhido)
    database.insert_wallet(data=wallet)
database.con.close()

### Gerando registros para a tabela Trade

In [11]:
def gerador_hash_id():
    hash_id = ""
    for item in range(65):
        x = random.choice(string.ascii_uppercase + string.digits)
        hash_id += x
    return hash_id

def gerador_movimentacao():
    tipo = ("Debito", "Credito")
    retorno = random.choice(tipo)
    return retorno

def gerador_data():
        ano = random.randint(2023, 2023)
        mes = random.randint(1, 12)

        if mes == 1 or mes == 3 or mes == 5 or mes == 7 or mes == 8 or mes == 10 or mes == 12:
            dia = random.randint(1, 31)
        elif mes == 4 or mes == 6 or mes == 9 or mes == 11:
            dia = random.randint(1, 30)
        elif mes == 2:
            dia = random.randint(1, 28)
        return f"{ano}-{mes}-{dia}"
  
def gerador_hora():
    hora = random.randint(0, 23)
    minuto = random.randint(1, 59)
    segundo = random.randint(1, 60)
    return f"{hora}:{minuto}:{segundo}"

In [12]:
# Criando a conexão com o banco.
database = ConnectDB()
# Alterar a quantidade de repetições
QTD_REGISTROS = 1
for item in range (QTD_REGISTROS):
    # Criando o registro para a inserção
    trade = (gerador_hash_id(), gerador_movimentacao(), gerador_data(), gerador_hora())
    # Inserindo um registro tabela.
    database.insert_trade(data=trade)

# Fechando conexão com o banco.
database.con.close()

### Gerando registros para a tabela Criptomoeda

In [13]:
criptos =  [
            {'nome_cripto':'Bitcoin', 'sigla_cripto':'BTC', 'valor_atual':137884.01},
            {'nome_cripto':'Ethereum', 'sigla_cripto':'ETH', 'valor_atual':8693.45},
            {'nome_cripto': 'Ripple', 'sigla_cripto': 'XRP', 'valor_atual': 0.673},
            {'nome_cripto': 'Bitcoin Cash', 'sigla_cripto': 'BCH', 'valor_atual': 469.29},
            {'nome_cripto': 'Cardano', 'sigla_cripto': 'ADA', 'valor_atual': 1.56},
            {'nome_cripto': 'Binance Coin', 'sigla_cripto': 'BNB', 'valor_atual': 342.84},
            {'nome_cripto': 'Litecoin', 'sigla_cripto': 'LTC', 'valor_atual': 126.05},
            {'nome_cripto': 'Chainlink', 'sigla_cripto': 'LIN', 'valor_atual': 21.35},
            {'nome_cripto': 'Polkadot', 'sigla_cripto': 'DOT', 'valor_atual': 17.79},
            {'nome_cripto': 'Stellar', 'sigla_cripto': 'XLM', 'valor_atual': 0.276},
            {'nome_cripto': 'Dogecoin', 'sigla_cripto': 'DOG', 'valor_atual': 0.202},
            {'nome_cripto': 'Uniswap', 'sigla_cripto': 'UNI', 'valor_atual': 21.95},
            {'nome_cripto': 'VeChain', 'sigla_cripto': 'VET', 'valor_atual': 0.086},
            {'nome_cripto': 'Solana', 'sigla_cripto': 'SOL', 'valor_atual': 43.78},
            {'nome_cripto': 'Filecoin', 'sigla_cripto': 'FIL', 'valor_atual': 54.92},
            {'nome_cripto': 'Cosmos', 'sigla_cripto': 'ATO', 'valor_atual': 9.71},
            {'nome_cripto': 'Monero', 'sigla_cripto': 'XMR', 'valor_atual': 238.76},
            {'nome_cripto': 'TRON', 'sigla_cripto': 'TRX', 'valor_atual': 0.056},
            {'nome_cripto': 'EOS', 'sigla_cripto': 'EOS', 'valor_atual': 2.80},
            {'nome_cripto': 'Neo', 'sigla_cripto': 'NEO', 'valor_atual': 42.05},
            {'nome_cripto': 'IOTA', 'sigla_cripto': 'MIO', 'valor_atual': 0.85},
            {'nome_cripto': 'Dash', 'sigla_cripto': 'DAS', 'valor_atual': 118.32},
            {'nome_cripto': 'Zcash', 'sigla_cripto': 'ZEC', 'valor_atual': 107.45},
            ]


In [112]:
database = ConnectDB()
for moeda in criptos:
    registro = (moeda['nome_cripto'], moeda['sigla_cripto'], moeda['valor_atual'])
    database.insert_criptomoeda(registro)
database.con.close()

### Gerando registros para a tabela Criptos_na_carteira e Trade_realizada

In [14]:
def wallet_by_cliente():
    database = ConnectDB()
    cliente = random.choice(cpfs_clientes)
    lista_wallets_por_cliente = database.find_wallet_by_cliente(cliente)
    database.con.close()
    if(lista_wallets_por_cliente is None):
        #print("lista_wallets_por_cliente é nula")
        return {'cliente': 'nulo', 'wallet': 'nulo'}
    elif (len(lista_wallets_por_cliente) == 0):
        #print("lista_wallets_por_cliente é vazia")
        return {'cliente': 'nulo', 'wallet': 'nulo'}
    else:
        wallet = random.choice(lista_wallets_por_cliente)
        return {'cliente': cliente, 'wallet': wallet}
    

In [19]:
# Mudar a celula de cima, criar uma trade realizada para cada trade, sortear uma criptomoeda e um cliente, e verificar se o cliente tem wallets, se tiver sortear uma wallet
# Preencher a tabela cripto na carteira daquela wallet com base nas informações geradas a cima
database = ConnectDB()
lista_trade = database.lista_trade()
lista_criptos = database.lista_criptomoeda()

# Este loop abaixo executara uma transação para cada registro no objeto trade, note que, como este código executa muitas consultas, ele naturalmente é mais lento que os demais scripts
# por isso criei no final do código um if "de parada" caso não se deseje a execução completa do loop
CONDICAO_DE_PARADA = 0
i = 0
for trade_hash_id in lista_trade:

    cliente_wallet = wallet_by_cliente()

    if(cliente_wallet['cliente'] != 'nulo'):
        criptomoeda = random.choice(lista_criptos)
        registro_trade_realizada = (trade_hash_id, criptomoeda, cliente_wallet['wallet'], cliente_wallet['cliente'])
        database.insert_trade_realizada(registro_trade_realizada)
        registro_cripto_carteira = (random.randint(1, 10), criptomoeda, cliente_wallet['wallet'])
        database.insert_cripto_na_carteira(registro_cripto_carteira)

    ## Condição de parada
    #if i == CONDICAO_DE_PARADA:
    #    break
    #i = i + 1

database.con.close()


1
2
3
4
5
6
7
8
9
10
